In [82]:
import multiprocess
import time
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

kernels = ['linear', 'poly', 'rbf', 'sigmoid']
# penalización  = list(np.arange(0.1, 1, 0.1)) + list(range(1, 51)) + list(range(100, 501, 50))
penalización  = list(np.arange(0.1, 0.5, 0.1))

# tols =  [1e-3, 1e-4, 1e-5]
tols =  [1e-3]

gammas = ['scale', 'auto']

hyperparameters = []
for kernel in kernels:
    for c in penalización :
        for tol in tols :
            for gamma in gammas:
                hyperparameters.append([kernel, c, tol,gamma])

print(len(hyperparameters))
iris=datasets.load_wine()
X=iris.data
y=iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.20)

def evaluate_set( X_train, X_test, y_train, y_test, parametros, p_id, lock):
    from sklearn.svm import SVC
    from sklearn.metrics import accuracy_score
    import datetime
    print('\n Yo soy el proceso:', p_id, 'Comence a las:',datetime.datetime.now())
    for s in parametros:
        clf=SVC(kernel=(s[0]), C=float(s[1]), tol=float(s[2]), gamma=(s[3]))
        clf.fit(X_train, y_train)
        y_pred=clf.predict(X_test)
        lock.acquire()
        print(f'Proceso {p_id} con parametros {s} y accuracy {accuracy_score(y_test,y_pred)}')
        lock.release()

threads=[]
N_THREADS=4
splits=np.split(np.array(hyperparameters), N_THREADS)
lock=multiprocess.Lock()
for i in range(N_THREADS):
    # Se generan los hilos de procesamiento
    threads.append(multiprocess.Process(target=evaluate_set, args=[X_train,X_test,y_train,y_test, splits[i],i, lock]))

start_time = time.perf_counter()
# Se lanzan a ejecución
for thread in threads:
    thread.start()
# y se espera a que todos terminen
for thread in threads:
    thread.join()
finish_time = time.perf_counter()
print(f"Program finished in {finish_time-start_time} seconds")

32

 Yo soy el proceso:

 0
 Yo soy el proceso:  Comence a las:
 Yo soy el proceso:1   Comence a las:2024-04-16 11:30:49.5986662 
 Yo soy el proceso: 
 2024-04-16 11:30:49.616482Comence a las: 32024-04-16 11:30:49.633253
 
Comence a las: 2024-04-16 11:30:49.653592Proceso 1 con parametros ['poly' '0.1' '0.001' 'scale'] y accuracy 0.6944444444444444

Proceso 2 con parametros ['rbf' '0.1' '0.001' 'scale'] y accuracy 0.6944444444444444
Proceso 3 con parametros ['sigmoid' '0.1' '0.001' 'scale'] y accuracy 0.3888888888888889
Proceso 2 con parametros ['rbf' '0.1' '0.001' 'auto'] y accuracy 0.3888888888888889
Proceso 3 con parametros ['sigmoid' '0.1' '0.001' 'auto'] y accuracy 0.3888888888888889
Proceso 2 con parametros ['rbf' '0.2' '0.001' 'scale'] y accuracy 0.6944444444444444
Proceso 0 con parametros ['linear' '0.1' '0.001' 'scale'] y accuracy 0.9444444444444444
Proceso 3 con parametros ['sigmoid' '0.2' '0.001' 'scale'] y accuracy 0.3888888888888889
Proceso 2 con parametros ['rbf' '0.2' '0.001' 'auto'] y accuracy 0

In [70]:
import multiprocess
import time
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

def nivelacion_cargas(D, n_p):
    s=len(D)%n_p
    t=int((len(D)-s)/n_p)
    out=[]
    for i in range(n_p):
        if i<s:
            out.append(D[i*t+i:i*t+i+t+1])            
        else:
            out.append(D[i*t+s:i*t+s+t])
    return out

def evaluate_set( X_train, X_test, y_train, y_test, parametros, p_id, lock):
    from sklearn.svm import SVC
    from sklearn.metrics import accuracy_score
    import datetime
    print('\n Yo soy el proceso:', p_id, 'Comence a las:',datetime.datetime.now())
    for s in parametros:
        if s[0] == "linear":
            clf=SVC(kernel=(s[0]), C=float(s[1]), tol=float(s[2]))
            clf.fit(X_train, y_train)
            y_pred=clf.predict(X_test)
            lock.acquire()
            print(f'Proceso {p_id} con parametros {s} y accuracy {accuracy_score(y_test,y_pred)}')
            lock.release()



kernels = ['linear', 'poly', 'rbf', 'sigmoid']
# penalización  = list(np.arange(0.1, 1, 0.1)) + list(range(1, 51)) + list(range(100, 501, 50))
penalización  = list(np.arange(0.1, 1, 0.1))

# tols =  [1e-3, 1e-4, 1e-5]
tols =  [1e-3]

gammas = ['scale', 'auto']

hyperparameters = []
for kernel in kernels:
    for c in penalización :
        for tol in tols :
            # for gamma in gammas:
            hyperparameters.append([kernel, c, tol])


iris=datasets.load_wine()
X=iris.data
y=iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.20)

threads=[]
N_THREADS=4
splits= nivelacion_cargas(np.array(hyperparameters),N_THREADS)
lock=multiprocess.Lock()

print(splits)
for i in range(N_THREADS):
    # Se generan los hilos de procesamiento
    threads.append(multiprocess.Process(target=evaluate_set, args=[X_train,X_test,y_train,y_test, splits[i],i, lock]))

start_time = time.perf_counter()
# Se lanzan a ejecución
for thread in threads:
    thread.start()
# y se espera a que todos terminen
for thread in threads:
    thread.join()
finish_time = time.perf_counter()
print(f"Program finished in {finish_time-start_time} seconds")       

[array([['linear', '0.1', '0.001'],
       ['linear', '0.2', '0.001'],
       ['linear', '0.30000000000000004', '0.001'],
       ['linear', '0.4', '0.001'],
       ['linear', '0.5', '0.001'],
       ['linear', '0.6', '0.001'],
       ['linear', '0.7000000000000001', '0.001'],
       ['linear', '0.8', '0.001'],
       ['linear', '0.9', '0.001']], dtype='<U32'), array([['poly', '0.1', '0.001'],
       ['poly', '0.2', '0.001'],
       ['poly', '0.30000000000000004', '0.001'],
       ['poly', '0.4', '0.001'],
       ['poly', '0.5', '0.001'],
       ['poly', '0.6', '0.001'],
       ['poly', '0.7000000000000001', '0.001'],
       ['poly', '0.8', '0.001'],
       ['poly', '0.9', '0.001']], dtype='<U32'), array([['rbf', '0.1', '0.001'],
       ['rbf', '0.2', '0.001'],
       ['rbf', '0.30000000000000004', '0.001'],
       ['rbf', '0.4', '0.001'],
       ['rbf', '0.5', '0.001'],
       ['rbf', '0.6', '0.001'],
       ['rbf', '0.7000000000000001', '0.001'],
       ['rbf', '0.8', '0.001'],
      

In [62]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

iris=datasets.load_wine()
X=iris.data
y=iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.20)

svm_classifier = SVC(kernel='linear', C=450, tol=0.001, gamma="scale")

svm_classifier.fit(X_train, y_train)

y_pred = svm_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo SVM: {accuracy}')


Precisión del modelo SVM: 1.0
